In [1]:
println("test")

test


# Download Dataset

In [ ]:
cd("../dataset") do
    run(`wget http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip`) 
    run(`unzip mini_speech_commands.zip`)
end

In [8]:
using Markdown

Markdown.parse_file("../dataset/mini_speech_commands/README.md")

# Speech Commands dataset excerpt

This is a small excerpt of the [Speech Commands Dataset](https://ai.googleblog.com/2017/08/launching-speech-commands-dataset.html) for use in a tutorial on tensorflow.org. Please refer to the original [dataset](http://download.tensorflow.org/data/speech*commands*v0.01.tar.gz](http://download.tensorflow.org/data/speech*commands*v0.01.tar.gz) for documentation and license information."""


In [9]:
readdir("../dataset/mini_speech_commands")

9-element Vector{String}:
 "README.md"
 "down"
 "go"
 "left"
 "no"
 "right"
 "stop"
 "up"
 "yes"

In [10]:
readdir("../dataset/mini_speech_commands/down")

1000-element Vector{String}:
 "004ae714_nohash_0.wav"
 "00b01445_nohash_1.wav"
 "00f0204f_nohash_0.wav"
 "0132a06d_nohash_1.wav"
 "0132a06d_nohash_4.wav"
 "0137b3f4_nohash_2.wav"
 "014f9f65_nohash_0.wav"
 "016e2c6d_nohash_0.wav"
 "016e2c6d_nohash_2.wav"
 "01b4757a_nohash_0.wav"
 ⋮
 "fda46b78_nohash_1.wav"
 "fe291fa9_nohash_0.wav"
 "fe5c4a7a_nohash_1.wav"
 "feb1d305_nohash_0.wav"
 "feb1d305_nohash_1.wav"
 "ff21fb59_nohash_0.wav"
 "ff63ab0b_nohash_0.wav"
 "ffd2ba2f_nohash_0.wav"
 "ffd2ba2f_nohash_4.wav"